<a href="https://colab.research.google.com/github/seonhong416/Job-Care/blob/kimseonhong/job_care_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 모듈 불러오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('/content/drive/MyDrive/jobcare/train_data_1.csv')

In [3]:
data

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,month,date,hour,day
0,1,1,1,0,0,0,1,4,3,5,275,370,369,8,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,3,316,94,398,58,368,94,422,1607,1606,1605,2016,1,17,12,4
1,0,0,0,1,1,0,1,3,4,1,114,181,175,4,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,94,417,94,400,94,399,94,417,1607,1606,1605,2016,6,18,17,3
2,0,0,0,1,0,0,2,0,3,5,464,175,452,3,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,48,364,250,528,48,366,48,363,1599,1595,1572,2016,7,8,20,2
3,0,0,0,1,0,0,2,0,2,5,703,705,704,3,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,71,379,226,495,1,315,71,381,1607,1606,1605,2016,1,13,18,0
4,1,1,1,0,0,0,1,3,4,5,275,370,369,4,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,208,483,208,481,208,480,71,381,1607,1606,1605,2016,3,9,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,0,0,0,1,0,0,1,1,2,2,1192,935,1228,3,59,4,95,3,3,5,1,1,2,354,147,1,5,65,503156,285850,1,1192,1180,1169,926,929,928,927,926,1227,1227,1227,926,145,123,56,1,58,368,3,316,94,398,58,373,352,347,346,2006,3,13,12,4
501947,1,1,0,1,0,0,1,6,2,1,118,113,110,4,105,142,95,3,3,10,2,1,2,163,120,1,4,142,676255,456996,1,114,109,56,1,110,109,56,1,110,109,56,1,114,109,56,1,94,401,94,425,94,398,94,425,155,138,99,2006,1,20,11,0
501948,1,1,1,1,0,0,1,7,4,1,147,46,145,4,59,127,139,3,1,5,1,1,2,438,147,2,7,65,484528,293258,1,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,58,368,94,413,94,422,58,373,437,417,407,2006,8,5,17,2
501949,1,0,0,1,0,0,1,1,2,1,46,147,145,4,251,49,258,3,2,5,1,1,2,660,147,3,4,259,456330,273797,1,46,43,2,1,145,123,56,1,145,123,56,1,145,123,56,1,250,518,48,359,250,523,250,524,659,652,651,2006,6,15,9,0


# 테스트가 imbalance한 데이터를 어떻게 처리해야될까?

train과 test를 imbalance 하게 만들어서 학습 시켜보자


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [5]:
train_, test_ = train_test_split(data, test_size = 0.2, random_state = 42)

In [6]:
train_['target'].value_counts()

0    200947
1    200613
Name: target, dtype: int64

In [7]:
test_['target'].value_counts()

1    50232
0    50159
Name: target, dtype: int64

In [8]:
train_0 = train_.loc[train_['target'] == 0]
train_1 = train_.loc[train_['target'] == 1]

In [9]:
test_0 = test_.loc[test_['target'] == 0]
test_1 = test_.loc[test_['target'] == 1]

In [10]:
train_0_train, train_0_test = train_test_split(train_0, test_size = 0.1, random_state = 42)

In [11]:
test_0_train, test_0_test = train_test_split(test_0, test_size = 0.1, random_state = 42)

In [12]:
train_0_train

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,month,date,hour,day
260445,1,0,0,1,1,1,1,4,1,1,998,1228,1084,0,67,64,66,3,3,5,1,1,2,561,1227,1,3,67,506723,308382,0,998,996,927,926,1227,1227,1227,926,1084,1083,1053,926,1227,1227,1227,926,58,375,58,372,58,374,58,375,561,544,527,2006,6,30,5,1
113011,0,0,0,0,0,0,1,5,2,3,841,102,705,6,4,127,59,2,3,5,1,3,2,1648,899,1,4,127,644777,3335,0,840,831,824,744,101,91,56,1,703,703,690,618,898,884,864,864,3,316,94,413,58,368,94,413,1647,1646,1645,2017,4,6,11,0
176446,1,0,0,1,1,1,1,5,3,1,114,181,175,4,114,95,96,3,3,3,1,3,2,1805,104,4,3,114,745440,516413,0,114,109,56,1,175,152,56,1,175,152,56,1,101,91,56,1,94,405,94,398,94,399,94,405,1803,1783,1752,2019,3,4,19,2
230476,1,1,0,1,0,0,1,5,2,1,1227,718,726,3,42,47,291,3,3,4,1,3,2,441,1228,2,3,47,164094,91993,0,1227,1227,1227,926,718,717,716,618,724,717,716,618,1227,1227,1227,926,30,353,30,358,277,550,30,358,441,441,441,2006,1,7,11,1
723,0,0,0,1,1,0,2,0,3,1,1160,136,95,4,119,4,4,3,2,5,1,1,2,692,316,1,4,120,774218,537564,0,1160,1158,1137,926,136,123,56,1,92,91,56,1,316,315,297,216,94,408,3,316,3,316,94,408,688,687,651,2006,6,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257357,0,0,0,1,0,0,1,0,2,1,1084,1085,1086,5,133,131,101,3,2,5,1,3,2,996,872,3,5,139,571064,580611,0,1084,1083,1053,926,1085,1083,1053,926,1086,1086,1053,926,872,865,864,864,94,417,94,417,94,400,94,422,993,987,954,2009,7,2,8,3
143176,1,1,1,1,1,0,1,3,2,1,149,98,102,4,190,189,114,3,1,10,2,1,2,561,149,1,7,192,1009992,714380,0,145,123,56,1,97,91,56,1,101,91,56,1,145,123,56,1,188,464,188,463,94,405,188,464,561,544,527,2006,2,25,7,1
136457,1,0,0,1,1,1,1,6,3,1,929,989,935,5,263,56,49,3,3,5,1,1,2,913,1132,1,4,263,386796,230396,0,929,928,927,926,989,988,927,926,929,928,927,926,1130,1129,1104,926,250,528,48,366,48,359,250,528,912,909,901,2008,1,22,17,2
88134,0,0,0,1,0,0,2,0,3,1,955,997,1085,3,72,227,2,1,3,3,1,1,2,1851,452,4,3,74,836963,578614,0,955,951,927,926,997,996,927,926,1085,1083,1053,926,452,451,450,377,71,379,226,495,1,315,71,381,1846,1843,1832,2020,5,27,8,2


In [13]:
train_0_test

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,month,date,hour,day
251093,0,0,0,1,1,0,1,5,3,2,447,101,446,3,279,278,288,2,3,5,1,3,2,1663,181,1,3,281,247437,83528,0,446,446,439,377,101,91,56,1,446,446,439,377,175,152,56,1,277,543,277,542,277,547,277,543,1662,1659,1658,2017,10,7,21,2
281166,0,0,0,0,0,0,1,4,4,1,333,149,299,0,2,4,95,3,1,5,1,1,2,992,146,1,5,77,869952,2568,0,333,332,297,216,145,123,56,1,299,298,297,216,145,123,56,1,1,315,3,316,94,398,71,384,991,987,954,2009,3,16,15,0
264092,0,0,0,1,1,1,1,1,2,2,180,182,175,4,229,72,227,1,3,10,2,2,2,1406,775,1,3,229,923886,651530,0,175,152,56,1,182,152,56,1,175,152,56,1,775,775,745,744,226,497,71,379,226,495,226,497,1403,1398,1397,2013,10,5,17,0
32248,0,0,0,0,0,0,2,0,5,5,459,454,465,3,2,4,95,3,3,5,1,3,2,1662,862,3,4,178,935572,31977,0,457,451,450,377,452,451,450,377,465,463,450,377,860,853,824,744,1,315,3,316,94,398,169,454,1662,1659,1658,2017,8,3,0,0
314896,1,0,0,1,0,0,1,3,2,5,95,102,93,4,114,139,96,1,3,5,1,1,2,420,114,1,4,139,750191,368037,0,92,91,56,1,101,91,56,1,92,91,56,1,114,109,56,1,94,405,94,422,94,399,94,422,418,417,407,2006,7,7,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232881,0,0,0,1,1,1,1,4,2,5,998,1090,1091,3,258,251,49,1,1,5,1,1,2,1687,102,1,5,258,439990,266971,0,998,996,927,926,1086,1086,1053,926,1086,1086,1053,926,101,91,56,1,250,523,250,518,48,359,250,523,1682,1677,1658,2017,6,28,9,6
114112,1,0,0,1,1,1,1,2,2,5,1070,1078,1082,0,262,251,257,3,3,5,1,1,2,670,1006,1,3,262,454220,276016,0,1070,1062,1053,926,1078,1078,1053,926,1078,1078,1053,926,1003,1001,1000,926,250,527,250,518,250,522,250,527,668,667,651,2006,9,17,22,3
54189,1,1,0,1,0,0,2,0,4,1,664,665,683,3,96,116,139,1,3,2,1,3,2,1755,665,1,3,139,555572,376953,0,662,660,659,618,662,660,659,618,683,677,659,618,662,660,659,618,94,399,94,407,94,422,94,422,1754,1753,1752,2019,10,28,14,2
79661,1,0,0,1,0,0,1,1,3,1,120,118,113,4,86,90,278,1,3,5,1,1,2,344,143,1,4,91,275190,152649,0,114,109,56,1,114,109,56,1,110,109,56,1,136,123,56,1,85,391,85,395,277,542,85,396,343,340,335,2006,7,16,12,3


In [14]:
train = pd.concat([train_1, train_0_test], axis = 0)
test = pd.concat([test_1, test_0_test], axis = 0)

In [15]:
train

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,month,date,hour,day
470805,1,0,0,0,0,0,1,5,5,5,238,246,255,8,93,79,189,3,3,5,1,1,2,642,243,1,4,83,967249,662794,1,238,238,231,216,246,242,231,216,254,242,231,216,243,242,231,216,92,397,78,385,188,463,78,389,635,634,623,2006,1,14,15,1
9771,0,0,0,1,1,1,2,0,4,5,857,840,837,4,263,49,49,2,2,10,2,1,2,759,963,1,5,263,386980,231122,1,854,853,824,744,840,831,824,744,836,831,824,744,962,961,927,926,250,528,48,359,48,359,250,528,759,759,759,2006,2,20,17,3
490173,1,1,1,0,0,0,1,7,4,5,299,372,149,0,4,95,278,3,1,5,1,1,2,706,299,1,7,280,44163,140448,1,299,298,297,216,369,368,297,216,145,123,56,1,299,298,297,216,3,316,94,398,277,542,277,543,704,694,693,2006,10,9,19,4
434187,1,1,0,0,0,0,1,4,3,1,136,144,46,5,118,65,116,2,3,5,1,1,2,408,143,1,4,65,666510,292758,1,136,123,56,1,136,123,56,1,46,43,2,1,136,123,56,1,94,407,58,373,94,407,58,373,408,408,407,2006,7,20,22,0
319581,0,0,0,1,1,1,2,0,2,1,514,114,122,3,151,150,95,3,3,5,1,1,2,1077,102,1,3,151,798165,553406,1,514,509,482,482,114,109,56,1,114,109,56,1,101,91,56,1,149,433,149,432,94,398,149,433,1077,1060,1021,2010,2,3,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232881,0,0,0,1,1,1,1,4,2,5,998,1090,1091,3,258,251,49,1,1,5,1,1,2,1687,102,1,5,258,439990,266971,0,998,996,927,926,1086,1086,1053,926,1086,1086,1053,926,101,91,56,1,250,523,250,518,48,359,250,523,1682,1677,1658,2017,6,28,9,6
114112,1,0,0,1,1,1,1,2,2,5,1070,1078,1082,0,262,251,257,3,3,5,1,1,2,670,1006,1,3,262,454220,276016,0,1070,1062,1053,926,1078,1078,1053,926,1078,1078,1053,926,1003,1001,1000,926,250,527,250,518,250,522,250,527,668,667,651,2006,9,17,22,3
54189,1,1,0,1,0,0,2,0,4,1,664,665,683,3,96,116,139,1,3,2,1,3,2,1755,665,1,3,139,555572,376953,0,662,660,659,618,662,660,659,618,683,677,659,618,662,660,659,618,94,399,94,407,94,422,94,422,1754,1753,1752,2019,10,28,14,2
79661,1,0,0,1,0,0,1,1,3,1,120,118,113,4,86,90,278,1,3,5,1,1,2,344,143,1,4,91,275190,152649,0,114,109,56,1,114,109,56,1,110,109,56,1,136,123,56,1,85,391,85,395,277,542,85,396,343,340,335,2006,7,16,12,3


In [16]:
test

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,month,date,hour,day
42085,1,0,0,1,0,0,1,4,2,1,1227,1086,857,4,41,43,36,3,2,5,1,1,2,783,998,2,3,43,152543,106834,1,1227,1227,1227,926,1086,1086,1053,926,854,853,824,744,998,996,927,926,30,352,30,354,30,347,30,354,779,771,759,2006,2,7,15,4
494803,1,1,1,1,1,1,1,7,2,1,514,464,122,3,21,113,19,1,3,9,2,1,2,1835,514,4,3,21,113028,72790,1,514,509,482,482,464,463,450,377,114,109,56,1,514,509,482,482,3,333,94,404,3,331,3,333,1834,1833,1832,2020,11,10,12,1
283408,0,0,0,1,0,0,2,1,1,1,782,174,175,4,8,4,29,1,3,5,1,1,2,311,575,5,3,6,16775,13057,1,782,782,745,744,174,152,56,1,175,152,56,1,573,560,523,522,3,320,3,316,3,341,3,318,310,306,295,2006,3,23,15,0
33052,1,1,0,1,0,0,1,4,4,1,102,145,10,0,95,150,251,3,3,3,1,1,2,1918,106,4,4,98,548784,332151,1,101,91,56,1,145,123,56,1,8,6,2,1,101,91,56,1,94,398,149,432,250,518,94,399,1918,1918,1918,2022,6,9,16,1
434511,1,0,0,1,0,0,2,0,2,5,1227,1086,857,0,125,124,115,3,3,3,1,1,2,1658,1132,5,5,115,696372,468066,1,1227,1227,1227,926,1086,1086,1053,926,854,853,824,744,1130,1129,1104,926,94,411,94,410,94,406,94,406,1658,1658,1658,2017,3,11,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268523,1,1,1,1,0,0,1,4,3,5,102,181,95,4,272,263,56,3,1,10,2,1,2,841,102,1,4,271,368716,217203,0,101,91,56,1,175,152,56,1,92,91,56,1,101,91,56,1,250,537,250,528,48,366,250,536,838,837,826,2006,11,17,18,1
230922,0,0,0,1,1,1,2,1,4,1,102,1227,697,4,152,150,95,3,3,9,2,2,1,1809,662,4,1,152,814990,564792,0,101,91,56,1,1227,1227,1227,926,697,691,690,618,662,660,659,618,149,434,149,432,94,398,149,434,1807,1806,1752,2019,4,11,9,5
232664,0,0,0,1,0,0,2,0,2,5,573,581,575,4,72,227,2,3,3,4,1,3,2,1555,102,1,3,73,828906,571858,0,573,560,523,522,580,560,523,522,573,560,523,522,101,91,56,1,71,379,226,495,1,315,71,380,1552,1548,1538,2015,1,22,17,2
332844,0,0,0,0,0,0,2,0,2,1,1227,1086,857,6,86,31,278,1,1,5,1,1,2,553,861,1,4,255,258271,258941,0,1227,1227,1227,926,1086,1086,1053,926,854,853,824,744,860,853,824,744,85,391,30,342,277,542,250,520,550,544,527,2006,2,19,19,2


In [17]:
train.target.value_counts()

1    200613
0     20095
Name: target, dtype: int64

In [18]:
test.target.value_counts()

1    50232
0     5016
Name: target, dtype: int64

# Model

In [19]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [20]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [21]:
X_train = train.drop('target', axis = 1)
y_train = train['target']
X_test = test.drop('target', axis = 1)
y_test = test['target']

In [22]:
cat = CatBoostClassifier(random_state = 42)
xgb = XGBClassifier(random_state = 42)
lgb = LGBMClassifier(random_state = 42)
bag = BaggingClassifier(random_state = 42)
rfc = RandomForestClassifier(random_state = 42)
ada = AdaBoostClassifier(random_state = 42)
gbc = GradientBoostingClassifier(random_state = 42)

In [23]:
def modelling (model, X_train, X_test, y_train, y_test) :
  model.fit(X_train, y_train)
  pred = model.predict(X_train)
  pred_df = pd.DataFrame(pred, columns = ['target'])
  f1_train = f1_score(y_train, pred_df)
  y_pred = model.predict(X_test)
  y_pred_df = pd.DataFrame(y_pred, columns = ['target'])
  f1_test = f1_score(y_test, y_pred_df)
  return f1_train, f1_test

In [25]:

model_dict = {'CatBoostClassifier' : cat, 'XGBClassifier' : xgb, 'LGBMClassifier' : lgb, 'RandomForestClassifier' : rfc,
             'AdaBoostClassifier' : ada, 'BaggingClassifier' : bag, 'GradientBoostingClassifier' : gbc}



result_df = pd.DataFrame(columns = ['name', 'f1_train', 'f1_test'])

for key, val in model_dict.items() : 
  
  result = modelling(val, X_train, X_test, y_train, y_test)
  result_df = result_df.append({'name' :  key, 'f1_train' : result[0], 'f1_test' : result[1]}, ignore_index = True)

result_df = result_df.sort_values('f1_test', ascending = False)
result_df = result_df.reset_index()
result_df.drop('index', axis = 1, inplace = True)

Learning rate set to 0.103215
0:	learn: 0.6009828	total: 131ms	remaining: 2m 10s
1:	learn: 0.5321360	total: 210ms	remaining: 1m 44s
2:	learn: 0.4792565	total: 311ms	remaining: 1m 43s
3:	learn: 0.4389921	total: 385ms	remaining: 1m 35s
4:	learn: 0.4071947	total: 473ms	remaining: 1m 34s
5:	learn: 0.3833003	total: 553ms	remaining: 1m 31s
6:	learn: 0.3653300	total: 630ms	remaining: 1m 29s
7:	learn: 0.3514083	total: 701ms	remaining: 1m 26s
8:	learn: 0.3403436	total: 789ms	remaining: 1m 26s
9:	learn: 0.3313828	total: 881ms	remaining: 1m 27s
10:	learn: 0.3240880	total: 975ms	remaining: 1m 27s
11:	learn: 0.3186645	total: 1.05s	remaining: 1m 26s
12:	learn: 0.3143787	total: 1.14s	remaining: 1m 26s
13:	learn: 0.3106961	total: 1.22s	remaining: 1m 26s
14:	learn: 0.3077881	total: 1.3s	remaining: 1m 25s
15:	learn: 0.3054725	total: 1.37s	remaining: 1m 24s
16:	learn: 0.3037796	total: 1.45s	remaining: 1m 23s
17:	learn: 0.3022911	total: 1.53s	remaining: 1m 23s
18:	learn: 0.3010646	total: 1.62s	remaining: 

In [26]:
result_df

,name,f1_train,f1_test
0,CatBoostClassifier,0.954442,0.952936
1,LGBMClassifier,0.952825,0.952695
2,RandomForestClassifier,0.999978,0.952499
3,XGBClassifier,0.952454,0.952456
4,GradientBoostingClassifier,0.952473,0.952424
5,AdaBoostClassifier,0.952395,0.952395
6,BaggingClassifier,0.996641,0.947181


# 제출해보기

In [30]:
test_data = pd.read_csv('/content/drive/MyDrive/jobcare/test.csv',parse_dates=['contents_open_dt'])

d_code = pd.read_csv('/content/drive/MyDrive/jobcare/속성_D_코드.csv', index_col=0).T.to_dict()

h_code = pd.read_csv('/content/drive/MyDrive/jobcare/속성_H_코드.csv', index_col=0).T.to_dict()

l_code = pd.read_csv('/content/drive/MyDrive/jobcare/속성_L_코드.csv', index_col=0).T.to_dict()

In [28]:
test_data

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,46399,True,True,True,False,False,False,2,0,4,1,147,46,145,4,1,1,2,4,95,3,3,5,1,1,2,759,147,1,5,91,425040,726084,2020-12-17 01:06:26
46400,46400,True,False,False,True,False,False,2,0,4,1,176,120,159,4,1,1,86,31,278,3,3,10,2,1,2,759,147,1,4,91,290061,156948,2020-12-29 21:57:25
46401,46401,True,True,False,True,True,True,2,0,3,1,145,46,147,5,1,1,288,279,278,3,3,10,2,1,2,759,147,1,4,288,307951,175069,2020-12-10 19:09:09
46402,46402,True,True,False,True,True,True,1,3,3,2,145,46,147,6,1,1,288,279,278,3,1,5,1,3,2,759,147,1,5,288,308354,174849,2020-12-07 14:04:34


In [31]:
def add_code(df, d_code, h_code, l_code):
    
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df

test_data_1 = add_code(test_data, d_code, h_code, l_code)

In [32]:
test_data_1

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,250,528,48,366,48,359,250,528,1146,1128,1021,2010
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,250,523,250,528,48,359,250,528,1610,1606,1605,2016
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,169,453,169,451,169,452,169,453,1812,1811,1810,2020
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,169,453,169,451,169,452,169,453,101,100,99,2006
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,169,454,169,453,3,316,169,453,984,980,954,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,46399,True,True,True,False,False,False,2,0,4,1,147,46,145,4,1,1,2,4,95,3,3,5,1,1,2,759,147,1,5,91,425040,726084,2020-12-17 01:06:26,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,1,315,3,316,94,398,85,396,759,759,759,2006
46400,46400,True,False,False,True,False,False,2,0,4,1,176,120,159,4,1,1,86,31,278,3,3,10,2,1,2,759,147,1,4,91,290061,156948,2020-12-29 21:57:25,175,152,56,1,114,109,56,1,159,152,56,1,145,123,56,1,85,391,30,342,277,542,85,396,759,759,759,2006
46401,46401,True,True,False,True,True,True,2,0,3,1,145,46,147,5,1,1,288,279,278,3,3,10,2,1,2,759,147,1,4,288,307951,175069,2020-12-10 19:09:09,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006
46402,46402,True,True,False,True,True,True,1,3,3,2,145,46,147,6,1,1,288,279,278,3,1,5,1,3,2,759,147,1,5,288,308354,174849,2020-12-07 14:04:34,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006


In [33]:
for i in test_data_1.columns[1:7] :
  test_data_1[i+'_1'] = 0
for i in test_data_1.columns[1:7] :
  test_data_1.loc[test_data_1[i] == True, i+'_1'] = 1

for i in test_data_1.columns[1:7] : 
  test_data_1[i] = test_data_1[i+'_1']

test_data_1 = test_data_1.drop(['d_l_match_yn_1', 'd_m_match_yn_1', 'd_s_match_yn_1',
       'h_l_match_yn_1', 'h_m_match_yn_1', 'h_s_match_yn_1'], axis = 1)

In [34]:
test_data_1

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,1,0,0,1,1,1,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,250,528,48,366,48,359,250,528,1146,1128,1021,2010
1,1,0,0,0,1,0,0,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,250,523,250,528,48,359,250,528,1610,1606,1605,2016
2,2,1,0,0,1,1,1,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,169,453,169,451,169,452,169,453,1812,1811,1810,2020
3,3,1,0,0,1,1,1,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,169,453,169,451,169,452,169,453,101,100,99,2006
4,4,1,0,0,1,0,0,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,169,454,169,453,3,316,169,453,984,980,954,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,46399,1,1,1,0,0,0,2,0,4,1,147,46,145,4,1,1,2,4,95,3,3,5,1,1,2,759,147,1,5,91,425040,726084,2020-12-17 01:06:26,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,1,315,3,316,94,398,85,396,759,759,759,2006
46400,46400,1,0,0,1,0,0,2,0,4,1,176,120,159,4,1,1,86,31,278,3,3,10,2,1,2,759,147,1,4,91,290061,156948,2020-12-29 21:57:25,175,152,56,1,114,109,56,1,159,152,56,1,145,123,56,1,85,391,30,342,277,542,85,396,759,759,759,2006
46401,46401,1,1,0,1,1,1,2,0,3,1,145,46,147,5,1,1,288,279,278,3,3,10,2,1,2,759,147,1,4,288,307951,175069,2020-12-10 19:09:09,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006
46402,46402,1,1,0,1,1,1,1,3,3,2,145,46,147,6,1,1,288,279,278,3,1,5,1,3,2,759,147,1,5,288,308354,174849,2020-12-07 14:04:34,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006


In [35]:
test_data_1 = test_data_1.drop(['id', 'person_prefer_f', 'person_prefer_g'], axis = 1)

test_data_1['month']=test_data_1['contents_open_dt'].dt.strftime("%m")
test_data_1['date']=test_data_1['contents_open_dt'].dt.strftime("%d")
test_data_1['hour']=test_data_1['contents_open_dt'].dt.strftime("%H")
test_data_1['day']=test_data_1['contents_open_dt'].dt.strftime("%A")

test_data_1['month']=test_data_1['month'].astype(int)
test_data_1['date']=test_data_1['date'].astype(int)
test_data_1['hour']=test_data_1['hour'].astype(int)

In [36]:
test_data_1.loc[test_data_1['day'] == 'Monday', 'day'] = 0
test_data_1.loc[test_data_1['day'] == 'Tuesday', 'day'] = 1
test_data_1.loc[test_data_1['day'] == 'Wednesday', 'day'] = 2
test_data_1.loc[test_data_1['day'] == 'Thursday', 'day'] = 3
test_data_1.loc[test_data_1['day'] == 'Friday', 'day'] = 4
test_data_1.loc[test_data_1['day'] == 'Saturday', 'day'] = 5
test_data_1.loc[test_data_1['day'] == 'Sunday', 'day'] = 6

In [37]:
test_data_1 = test_data_1.drop('contents_open_dt', axis = 1)

In [38]:
test_data_1.day

0        1
1        3
2        3
3        3
4        4
        ..
46399    3
46400    1
46401    3
46402    0
46403    1
Name: day, Length: 46404, dtype: object

In [39]:
test_data_1.day = test_data_1.day.astype('int64')

In [40]:
test_data_1.to_csv('/content/drive/MyDrive/jobcare/test_data_1.csv', index = False)

In [41]:
cat_pred = cat.predict(test_data_1)

In [42]:
cat_pred = pd.DataFrame(cat_pred, columns = ['cat_pred'])

In [44]:
cat_pred.value_counts()

cat_pred
1           45904
0             500
dtype: int64

In [45]:
submission = pd.read_csv('/content/drive/MyDrive/jobcare/sample_submission.csv')
submission['target'] = cat_pred


In [46]:
submission.to_csv('/content/drive/MyDrive/jobcare/submission_cat.csv', index_label = False, index = False)

0.67305